In [ ]:
# 2023-05-23 - This code was created to quickly backup all of the good `loadedSessPickle.pkl` files from the session folders into the './output' subfolder in the session:
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/output/loadedSessPickle.pkl already in output folder.
# ...

In [1]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 3
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from copy import deepcopy
from datetime import datetime, timedelta
from typing import Optional, Dict, List


## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, FileList
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_files_metadata

# import glob # for finding .whl file after building binary repo
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun

global_data_root_parent_path = find_first_extant_path([Path(r'/media/MAX/Data'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'global_computation_results.pkl'

build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from pyphocorehelpers.gui.Jupyter.simple_widgets import build_global_data_root_parent_path_selection_widget
    
all_paths = [Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')]
global_data_root_parent_path = None
def on_user_update_path_selection(new_path: Path):
	global global_data_root_parent_path
	new_global_data_root_parent_path = new_path.resolve()
	global_data_root_parent_path = new_global_data_root_parent_path
	print(f'global_data_root_parent_path changed to {global_data_root_parent_path}')
	assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
			
global_data_root_parent_path_widget = build_global_data_root_parent_path_selection_widget(all_paths, on_user_update_path_selection)
global_data_root_parent_path_widget

global_data_root_parent_path changed to /home/halechr/FastData


ToggleButtons(description='Data Root:', layout=Layout(width='auto'), options=(PosixPath('/home/halechr/FastDat…

In [ ]:
global_data_root_parent_path

In [ ]:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

# BATCH_DATE_TO_USE = '2023-10-18_Apogee' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2023-11-15_Lab' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2024-01-09' # used for filenames throught the notebook


## Old (Pre 2023-09-21):

In [ ]:
# Use glob to find the first generated .whl file in the dist/ directory
found_whl_files = glob.glob('*.whl')
found_whl_files = [a for a in found_whl_files if not a.endswith('current.whl')] # exclude the symlink from the search

In [ ]:
external_computed_ripple_filepath = session.basepath.joinpath('ripple_df.pkl')
## try the '.ripple.npy' ripples:
active_file_suffix = '.ripple.npy'
external_computed_ripple_filepath = fp.with_suffix(active_file_suffix)



In [ ]:
## Moves all previously saved loadedSessPickle.pkl files to the output folder.
correctly_placed_output_files = []
search_path: Path = Path("/home/halechr/turbo/Data/KDIBA/").resolve()
assert search_path.exists()
for p in search_path.rglob(completed_pipeline_filename):
    if p.parent.name != 'output':
        # If not already in the 'output/' subfolder, move it there.
        print(f'p: {p}')
        curr_output_dir = p.parent.joinpath('output')
        curr_output_dir.mkdir(exist_ok=True)
        # print(f'curr_output_dir: {curr_output_dir}')
        new_path = p.replace(curr_output_dir.joinpath(p.name))
        print(f'\t new_path: {new_path}')
        correctly_placed_output_files.append(p)
    else:
        print(f'p: {p} already in output folder.')
        correctly_placed_output_files.append(p)
    
correctly_placed_output_files

In [ ]:
completed_pipeline_filename

In [ ]:

## Build Pickle Path:
pkl_path = 'global_batch_result_2023-06-08.pkl'
csv_path = 'global_batch_result_2023-06-08.csv'
h5_path = 'global_batch_result_2023-06-08.h5'

global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(h5_path).resolve() # Use Default

batch_progress_df = BatchRun.load_batch_progress_df_from_h5(global_batch_result_file_path)
batch_progress_df

updated_batch_progress_df = BatchRun.rebuild_basedirs(batch_progress_df, global_data_root_parent_path)
updated_batch_progress_df

updated_good_only_batch_progress_df = updated_batch_progress_df[updated_batch_progress_df['locally_is_ready']].copy()
updated_good_only_batch_progress_df

In [ ]:
# Generate code that can be pasted into the current "ReviewOfWork-*.ipynb" notebook to load the good sessions:
print(",\n".join([a_ctxt.get_initialization_code_string() for a_ctxt in updated_good_only_batch_progress_df['context'].to_list()]))


# 2023-09-21 - Mirror Slow Data files to much faster SSD: , Path(r'/home/halechr/FastData')

In [21]:
import os
from pathlib import Path
import shutil # used in `restore_symlink_folder`
from shutil import copytree # used in `make_specific_items_local`
from typing import Optional, Dict, List
from datetime import datetime, timedelta
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file
from attrs import define, field, Factory


In [46]:
def nearest_symlink_ancestor_path(path: Path) -> Path or None:
    """Returns the path of the nearest parent directory of 'path' that is a symbolic link.
    If no parent directories are symbolic links, returns None.
    
    Usage:
        existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/11-09_22-4-5/11-09_22-4-5IN.5.numclu')
        nearest_symlink_ancestor_path(existing_symlink_path)
        
    """

    for parent in path.parents:
        if parent.is_symlink():
            return parent
    return None

def symlinked_item_to_full_local_item(existing_symlink_path: Path, dryrun: bool=False) -> bool:
    """ takes a path to an existing symlink on disk. Replaces that symlink with a local file/folder of the same name, copied from the symlink target.

    returns bool: true if symlink was localized
        
    WARNING:
        DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
            
    Usage:

        existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/fet11-01_12-58-54') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
        print(f'existing_symlink_path: {existing_symlink_path}')
        assert existing_symlink_path.exists()
        symlinked_item_to_full_local_item(existing_symlink_path, dryrun=False)
        
        
    Potential Issue:
        if '/home/halechr/FastData/KDIBA/pin01' is a symlink to '/media/MAX/Data/KDIBA/pin01'
            If `symlinked_item_to_full_local_item` is called with existing_symlink_path='/home/halechr/FastData/KDIBA/pin01/one/fet11-01_12-58-54', doesn't it create a symlink to itself?


        '/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54'
    """
    # check that existing_symlink_path exists and is a symlink
    if existing_symlink_path.is_symlink():
        # Resolve the symlink without following it
        unlinked_target = existing_symlink_path.resolve(strict=False)
        # Check if the target's parent isn't under the symlink itself
        if unlinked_target.parent in existing_symlink_path.parents:
            print(f"Error: cannot copy a folder or file into its child '{existing_symlink_path}'!")
            return False
        

        # resolve the symlink and ensure it exists
        symlink_target = existing_symlink_path.resolve()
        if not symlink_target.exists():
            print(f"Symlink target {symlink_target} does not exist.")
            return False
        
        # delete the original symlink and replace with a copy of the symlink target
        if symlink_target.is_dir():        # if it's a directory
            if dryrun:
                print(f'Would delete symlink: {existing_symlink_path}')
                print(f'Would copy directory {symlink_target} to {existing_symlink_path}')
            else:
                os.remove(existing_symlink_path)
                shutil.copytree(symlink_target, existing_symlink_path)
        else:    # it's a file
            if dryrun:
                print(f'Would delete symlink: {existing_symlink_path}')
                print(f'Would copy file {symlink_target} to {existing_symlink_path}')
            else:
                os.remove(existing_symlink_path)
                shutil.copy(symlink_target, existing_symlink_path)

        return True
    else:
        print(f"{existing_symlink_path} is not a symlink.")
        
        existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/11-09_22-4-5/11-09_22-4-5IN.5.numclu')
        nearest_symlink_ancestor_path(existing_symlink_path)
        nearest_symlink_ancestor_path(existing_symlink_path)
        return False


def symlinked_folder_to_local_folder_containing_symlinks(existing_symlink_folder_path: Path, dryrun: bool=False) -> bool:
	""" takes a path to an existing symlink on disk. Replaces that symlink with a local folder it creates in the same place, and then creates symlinks inside the new folder to all items in the symlinked folder.

	returns bool: true if symlink was changed
		
	WARNING:
		DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
            
            
    Usage:
    
		existing_symlink_folder_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
		print(f'existing_symlink_folder_path: {existing_symlink_folder_path}')
		assert existing_symlink_folder_path.exists()
		symlinked_folder_to_local_folder_containing_symlinks(existing_symlink_folder_path, dryrun=False)

	"""
	# check that existing_symlink_folder_path exists, is a symlink, and links to a folder
	if existing_symlink_folder_path.is_symlink():
		destination_folder = existing_symlink_folder_path.resolve(strict=False)
		if destination_folder.is_dir():
			if dryrun:
				print(f'Would delete symlink: {existing_symlink_folder_path}')
				print(f'Would create directory: {existing_symlink_folder_path}')
				# print(f'Would create symlinks in {existing_symlink_folder_path} for all items in {destination_folder}')
			else:
				# delete the original symlink
				os.unlink(existing_symlink_folder_path)
				
				# create new folder where the symlink was
				os.mkdir(existing_symlink_folder_path)
				
			# create symlinks in the new folder for each file/folder in the original symlinked folder
			for item in destination_folder.iterdir():
				symlink_path = existing_symlink_folder_path / item.name
				if dryrun:
					print(f'\t Would create symlinks in {symlink_path}\t->\t{item}')
				else:	
					os.symlink(item, symlink_path)
					
			return True
		
		else:
			print(f"Original symlink destination {destination_folder} is not a directory.")
			return False

	else:
		print(f"{existing_symlink_folder_path} is not a symlink.")
		print(f'if this above shows an unexpected path (the target and not the symlink location you thought you passed in), make sure you are not calling `existing_symlink_folder_path.resolve()` on the input, this resolves the symlink!\n')
		return False


def restore_symlink_folder(original_folder_path: Path, dryrun: bool=False) -> bool:
    """
    This function takes a path to an existing directory filled with symlinked files and restores it to be one single symlink.
	Inverse of `symlinked_folder_to_local_folder_containing_symlinks`
	
	
    It works as follows:
    - Check if `original_folder_path` is a directory
    - If it is, scan its contents
    - All contents should be symbolic links. If not, print an error message/exit
    - All symbolic links should direct to the same directory. If not, print an error message/exit
    - With the target directory established, delete `original_folder_path` and all contents
    - Create a new symlink at `original_folder_path` pointing to the target directory
    
    
    Usage:
    
    	restore_symlink_folder(existing_symlink_folder_path, dryrun=False)
    
    """

    if original_folder_path.is_dir():
        # Collect all unique destinations this folder points to (should only be one)
        destinations = set(path.resolve().parent for path in original_folder_path.iterdir() if path.is_symlink()) # `set(...)` here is what make it so only unique entries are used

        if len(destinations) > 1:
            print(f'Error: multiple symlink destinations found in directory {original_folder_path}.\n\tdestinations: {destinations}')
            return False
        elif len(destinations) == 0:
            print(f'Error: no symlinks found in directory {original_folder_path}.')
            return False
        else:
            target_dir = destinations.pop()

            if dryrun:
                print(f'Would delete folder {original_folder_path} and its contents.')
                print(f'Would create symlink: {original_folder_path} -> {target_dir}')
            else:
                # delete the directory and its content
                shutil.rmtree(original_folder_path)

                # create the symlink
                os.symlink(target_dir, original_folder_path)
      
            return True
    else:
        print(f"Error: {original_folder_path} is not a directory.")
        return False
    


def make_specific_items_local(existing_symlink_folder_path: Path, desired_local_folder_paths: List[Path], dryrun: bool=False) -> bool:
    """ 
    Takes a path you wish to clone to a local folder from a symlink.
    It then proceeds to copy the directories from the source directory to the destination directory using `shutil.copytree()`
    This will handle going into subdirectories as well.

    existing_symlink_folder_path: a path to the symlinked folder.
    desired_local_folder_paths: a list of paths you wish to copy locally from the file.
    """
    if existing_symlink_folder_path.is_symlink():
        symlink_dest = existing_symlink_folder_path.resolve(strict=False)
        
        for desired_path in desired_local_folder_paths:
            if not desired_path.is_symlink() and desired_path.is_dir():
                rel_dest_path = existing_symlink_folder_path / desired_path.relative_to(symlink_dest)
                
                if dryrun:
                    print(f"Would copy contents of {desired_path} to {rel_dest_path}")
                else:
                    copytree(src=str(desired_path), dst=str(rel_dest_path))
                
        return True
    else:
        print(f"{existing_symlink_folder_path} is not a symlink.")
        return False
    
	


In [36]:

existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {existing_symlink_path}')
assert existing_symlink_path.exists()
symlinked_folder_to_local_folder_containing_symlinks(existing_symlink_path, dryrun=False)

existing_symlink_folder_path: /home/halechr/FastData/KDIBA/pin01


True

In [37]:
existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {existing_symlink_path}')
assert existing_symlink_path.exists()
symlinked_folder_to_local_folder_containing_symlinks(existing_symlink_path, dryrun=False)

existing_symlink_folder_path: /home/halechr/FastData/KDIBA/pin01/one


True

In [38]:
# make_specific_items_local

# existing_symlink_folder_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one')
print(f'existing_symlink_folder_path: {existing_symlink_path}')
assert existing_symlink_path.exists()

desired_local_folder_paths: List[Path] = [Path(v) for v in ['/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54']]

make_specific_items_local(existing_symlink_path, desired_local_folder_paths=desired_local_folder_paths, dryrun=True)


existing_symlink_folder_path: /home/halechr/FastData/KDIBA/pin01
/home/halechr/FastData/KDIBA/pin01 is not a symlink.


False

In [43]:
existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/fet11-01_12-58-54') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_path: {existing_symlink_path}')
assert existing_symlink_path.exists()
symlinked_item_to_full_local_item(existing_symlink_path, dryrun=False)


existing_symlink_path: /home/halechr/FastData/KDIBA/pin01/one/fet11-01_12-58-54


True

In [49]:
existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/11-09_22-4-5/11-09_22-4-5IN.5.numclu')
nearest_symlink_ancestor_path(existing_symlink_path)

PosixPath('/home/halechr/FastData/KDIBA/pin01/one/11-09_22-4-5')

In [34]:
existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {existing_symlink_path}')
assert existing_symlink_path.exists()
restore_symlink_folder(existing_symlink_path, dryrun=False)

existing_symlink_folder_path: /home/halechr/FastData/KDIBA/pin01


True

In [ ]:
def attribute_is_extant_path(instance, attribute, value):
	if not isinstance(value, Path):
		raise TypeError(f"value must be a pathlib.Path but type(value): {type(value)}")
	if not value.exists():
		raise ValueError(f"value must exists on the filesystem!")
	


@define(slots=False)
class DataRepository:
	short_name: str = field()
	host: str = field()
	root_path: Path = field()

	# def __attrs_post_init__(self):
	# 	assert root_path.exists()
		
		
	def try_find_files(self):
		# Find the files and build the movedicts:
		source_data_root = self.root_path.resolve()
		found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
		found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
		return found_session_pickle_files, found_global_computation_results_files

data_repos = {'MAX':DataRepository('MAX', host='LNX00052', root_path=Path(r'/media/MAX/Data')),
'FastData':DataRepository('FastData', host='LNX00052', root_path=Path(r'/home/halechr/FastData')),
'gen_scripts':DataRepository('gen_scripts', host='LNX00052', root_path=Path(r'/home/halechr/FastData/gen_scripts/')),
}


included_session_contexts = [
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
	  IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

data_repos_good_session_concrete_folders_dict = {}

data_repos
# run_kdiba_gor01_one_2006-6-08_14-26-15

In [ ]:
curr_good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(data_repos['MAX'].root_path.resolve(), included_session_contexts)
curr_good_session_concrete_folders

In [ ]:
included_session_contexts

In [ ]:
data_repos_good_session_concrete_folders_dict['FastData'] = ConcreteSessionFolder.build_concrete_session_folders(data_repos['FastData'].root_path.resolve(), included_session_contexts, allow_create_needed_dirs=True)
data_repos_good_session_concrete_folders_dict['FastData']

In [ ]:
good_session_concrete_folders

In [ ]:
# copy_dict = ConcreteSessionFolder.build_backup_copydict(data_repos_good_session_concrete_folders_dict['FastData'], backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['global_pkl'])
copy_dict_FastData = ConcreteSessionFolder.build_backup_copydict(data_repos_good_session_concrete_folders_dict['FastData'], backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['local_pkl', 'global_pkl'])
copy_dict_FastData

In [ ]:
moved_files_dict_FastData_files = copy_movedict(copy_dict_FastData)
moved_files_dict_FastData_files

In [ ]:
found_session_pickle_files, found_global_computation_results_files = data_repos['MAX'].try_find_files()
found_session_pickle_files

In [ ]:

source_data_root = Path(r'/media/MAX/Data')
dest_data_root = Path(r'/home/halechr/FastData')
assert source_data_root.exists(), f"source_data_root: {source_data_root} does not exist! Is the right computer's config commented out above?"
assert dest_data_root.exists(), f"dest_data_root: {dest_data_root} does not exist! Is the right computer's config commented out above?"

oldest_modified_date = (datetime.now() - timedelta(days=5))

In [ ]:
# Find the files and build the movedicts:
found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
file_movedict_session_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_session_pickle_files, only_files_newer_than=oldest_modified_date)
file_movedict_global_computation_results_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_global_computation_results_files, only_files_newer_than=oldest_modified_date)

### Actually perform copy operations. This will take a while

In [ ]:
moved_files_dict_session_pickle_files = copy_movedict(file_movedict_session_pickle_files)
moved_files_dict_session_pickle_files

# moved_files_dict = copy_files(filelist_source, filelist_dest)

In [ ]:
moved_files_dict_global_computation_results_pickle_files = copy_movedict(file_movedict_global_computation_results_pickle_files)
moved_files_dict_global_computation_results_pickle_files

# Various file formats:
'global_computation_results_2023-12-07_GL.pkl'
'20231012125859-global_computation_results.pkltmp'
"loadedSessPickle_2023-12-07_GL.pkl"
"backup-20231020190542-loadedSessPickle.pkl"
"loadedSessPickle_2023-10-25_BidirectionalOnly"

In [ ]:

# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/home/halechr/cloud/turbo/Data'), Path(r'W:\Data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}

In [ ]:
found_files_dict = {}

for a_ctxt, a_path in session_basedirs_dict.items():
	found_any_kind_session_pickle_files = discover_data_files(a_path, glob_pattern='*loadedSessPickle*.pkl', recursive=True)
	found_any_global_computation_results_files = discover_data_files(a_path, glob_pattern='output/*global_computation_results*.pkl', recursive=True)
	found_files_dict[a_ctxt] = (found_any_kind_session_pickle_files, found_any_global_computation_results_files)

found_files_dict


In [ ]:
def render_tree(found_files_dict):
    tree_str = ""
    for session, (session_files, global_files) in found_files_dict.items():
        tree_str += f"{session}\n"
        if session_files:
            tree_str += "  - Session Pickle Files:\n"
            for file in session_files:
                tree_str += f"    * {file}\n"
        if global_files:
            tree_str += "  - Global Computation Results Files:\n"
            for file in global_files:
                tree_str += f"    * {file}\n"
    return tree_str

# Example of usage
tree_representation = render_tree(found_files_dict)
print(tree_representation)

In [ ]:
import ipywidgets as widgets

def create_tree_view(found_files_dict):
    sessions_accordion = widgets.Accordion(children=[widgets.VBox() for _ in found_files_dict])
    for i, (session, (session_files, global_files)) in enumerate(found_files_dict.items()):
        session_accordion = sessions_accordion.children[i]
        
        # Convert paths to strings and create session pickle files accordion
        session_files_accordion = widgets.Accordion(children=[
            widgets.VBox([widgets.Label(str(file)) for file in session_files])
        ])
        session_files_accordion.set_title(0, 'Session Pickle Files')

        # Convert paths to strings and create global computation results files accordion
        global_files_accordion = widgets.Accordion(children=[
            widgets.VBox([widgets.Label(str(file)) for file in global_files])
        ])
        global_files_accordion.set_title(0, 'Global Computation Results Files')

        # Set session accordion children
        session_accordion.children = [session_files_accordion, global_files_accordion]
        
        # Set main accordion title
        sessions_accordion.set_title(i, str(session))

    return sessions_accordion

# Example usage
tree_view = create_tree_view(found_files_dict)
tree_view


In [ ]:

# search_parent_path = Path(r'W:\\Data\\Kdiba')
# search_parent_path = Path(r'/home/halechr/turbo/Data/KDIBA').resolve() # Greatlakes
search_parent_path = Path(r'/home/halechr/cloud/turbo/Data/KDIBA').resolve() # Lab

# found_autoversioned_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*-loadedSessPickle.pkl', recursive=True)
# found_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*.pkl', recursive=True)


found_any_kind_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*loadedSessPickle*.pkl', recursive=True)
found_any_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*global_computation_results*.pkl', recursive=True)

# found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
# found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)

found_files = found_any_global_computation_results_files + found_any_kind_session_pickle_files
found_files

# 2023-11-20 - Discovery of folder modification times from matching files

In [ ]:
Write a python function that


In [ ]:

def determine_directory_modification_time(target_